In [1]:
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredPDFLoader
from REMOVED_SECRET import Document as LangchainDocument
from langchain_community.vectorstores import FAISS
from REMOVED_SECRET import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import torch
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, TextStreamer
from ragatouille import RAGPretrainedModel
import nltk
from transformers import AutoTokenizer, AutoModelForCausalLM
from rag_for_notebook_sunday import CustomTextGenerationPipeline
import REMOVED_SECRET as F

In [7]:
custom_pipeline = CustomTextGenerationPipeline(model_id="microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [8]:
prompt = "Hello, how are you?"

generated_sequence, all_logits = custom_pipeline(prompt)


#check if logits require gradients
print(f"Logits requires_grad: {all_logits.requires_grad}")



/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


How was your day? Assistant: I'm doing well—
Token_Logits tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
       requires_grad=True)
Not all logits are -inf
Token_Logits tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
       requires_grad=True)
Not all logits are -inf
Token_Logits tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
       requires_grad=True)
Not all logits are -inf
Token_Logits tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
       requires_grad=True)
Not all logits are -inf
Token_Logits tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
       requires_grad=True)
Not all logits are -inf
Token_Logits tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
       requires_grad=True)
Not all logits are -inf
Token_Logits tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
       requires_grad=True)
Not all logits are -inf
Token_Logits tens

In [9]:
import torch
import REMOVED_SECRET as F
print(f"all_logits shape: {all_logits.shape}")
# Assuming all_logits is your logits tensor
#batch_size, sequence_length, vocab_size = all_logits.size()
sequence_length, vocab_size = all_logits.size()
# Example target tensor: must be of shape (batch_size * sequence_length)
# Replace this with your actual target IDs. Here, we're simulating with random targets
target = torch.randint(0, vocab_size, (sequence_length,)).to(all_logits.device)
# Flatten the logits for loss computation
#logits_for_loss = all_logits.view(-1, vocab_size)  # shape: (batch_size * sequence_length, vocab_size)
#print(f"logits_for_loss shape: {logits_for_loss.shape}")
print(f"target shape: {target.shape}")
# Calculate the cross-entropy loss
loss = F.cross_entropy(all_logits, target)

# Backpropagation to compute gradients
loss.backward()
print("Loss:", loss)
print(f"Gradients computed for logits: {all_logits.grad is not None}")
print("Grads:", all_logits.grad)




all_logits shape: torch.Size([1, 448896])
target shape: torch.Size([1])
Loss: tensor(inf, device='cuda:0', grad_fn=<NllLossBackward0>)
Gradients computed for logits: True
Grads: tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
model_id = "microsoft/Phi-3-mini-4k-instruct"  # Or any other model you want to test
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                              torch_dtype=torch.float16, 
                                              device_map='auto', 
                                              attn_implementation="flash_attention_2", 

                                              trust_remote_code=True
                                             )
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create the text generation pipeline
generator = pipeline('text-generation', 
                     model=model, 
                     tokenizer=tokenizer, 
                     return_full_text=False, # To avoid including the prompt in the output
                     output_scores=True,    # To get the logits
                     max_new_tokens=50      # Adjust as needed
                    ) 

# Example prompt
prompt = "What is the capital of France?"

# Generate text and extract logits
with torch.no_grad():  # No need for gradients during inference
    outputs = generator(prompt)

generated_text = outputs[0]['generated_text'] 
logits = outputs[0]['scores']  # Access the logits

print("Generated Text:", generated_text)
print("Logits Shape:", logits.shape)  # Should be (sequence_length, vocab_size)